In [3]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2b5_AN_1FC_fold1_3.h5' #เปลี่ยน model ต้องเอา R2 ## 
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [6]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [7]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Traindf_fold1_3.csv') #เปลี่ยน Dataset
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(7392, 22)
(7392, 22)
Normal:  (4004, 22)
Abnormal:  (3388, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,0,1,42,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,231.0,134.0,544.0,380.0,0.229956,0.149621,0.541516,0.422222,AB02 P1 C042.JPG
1,1,6,42,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,229.0,98.0,645.0,550.0,0.228564,0.109217,0.643761,0.612121,AB04 P1 C042.JPG
2,2,10,42,P1,P1,Abnormal,AB06,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,267.0,45.0,564.0,525.0,0.263207,0.050631,0.555357,0.583838,AB06 P1 C042.JPG
3,3,20,22,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,200.0,141.0,589.0,501.0,0.201802,0.156841,0.592793,0.557769,AB01 P1 C022.JPG
4,4,29,22,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,155.0,67.0,506.0,639.0,0.166464,0.074874,0.543860,0.711111,AB02 P1 C022.JPG


In [6]:
# dataframe.info()

In [8]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 7392 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [9]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [10]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.11288008, -0.18320042,  0.21629888, ..., -0.16971584,
         0.07620948, -0.16377571],
       [-0.0767106 , -0.14782166,  0.46124855, ..., -0.09806491,
         0.20912299, -0.06951826],
       [ 0.75731391, -0.15539676,  0.30825716, ..., -0.15498731,
         0.06637455, -0.18102419],
       ...,
       [-0.16143344,  0.40907985, -0.18200816, ...,  0.49736023,
        -0.14770335,  0.53768939],
       [-0.02446694,  0.59848177, -0.14188574, ...,  0.57025063,
        -0.18778317, -0.05460552],
       [-0.15553439,  0.09028433, -0.22400035, ...,  0.30981982,
        -0.20732765,  0.35869294]])

In [11]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.112880,-0.183200,0.216299,0.254406,-0.144071,0.337477,0.332848,-0.178598,-0.209764,0.087514,...,-0.191967,0.203467,-0.204061,-0.200696,-0.168076,0.243658,-0.121029,-0.169716,0.076209,-0.163776
1,-0.076711,-0.147822,0.461249,0.225885,-0.193500,0.138018,0.472570,-0.209146,-0.204889,-0.019481,...,-0.126809,0.268982,-0.199199,-0.161693,-0.194916,0.395928,0.232385,-0.098065,0.209123,-0.069518
2,0.757314,-0.155397,0.308257,0.745639,-0.176845,0.635605,0.386733,-0.157212,-0.200776,0.120661,...,-0.179784,0.300050,-0.158562,-0.149302,-0.192254,0.285801,0.524784,-0.154987,0.066375,-0.181024
3,-0.084495,0.048628,0.265085,0.121334,-0.187050,0.025729,0.104737,-0.067526,-0.168360,0.039968,...,0.008038,-0.018834,-0.142258,-0.126904,-0.106197,-0.109417,-0.192129,-0.012116,-0.053060,-0.124021
4,0.285680,-0.117711,0.001491,0.253031,-0.234893,-0.061371,0.075303,-0.228541,-0.224167,-0.044950,...,-0.163440,-0.087389,-0.232703,-0.118822,-0.169264,-0.098076,-0.038218,-0.140207,-0.070978,-0.144560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,-0.072310,0.340018,-0.024193,0.037278,0.411343,-0.083988,-0.120341,0.151559,0.555141,-0.174963,...,0.414301,-0.194260,0.247091,0.199157,0.446409,-0.181699,-0.175733,0.433471,-0.087821,0.291353
7388,-0.092394,0.355854,-0.138060,-0.116591,0.532154,-0.201658,-0.204159,0.209273,0.526084,-0.185444,...,0.261122,-0.209993,0.116272,-0.078527,0.268936,-0.162285,-0.206996,0.732153,-0.185587,0.376708
7389,-0.161433,0.409080,-0.182008,0.004676,0.332312,-0.157609,-0.202943,0.292184,0.553160,-0.200821,...,0.991572,-0.149132,0.380707,0.252706,0.835365,-0.212773,-0.180168,0.497360,-0.147703,0.537689
7390,-0.024467,0.598482,-0.141886,-0.126161,0.181843,-0.185020,-0.187077,0.210121,0.048996,-0.164328,...,0.015964,-0.191710,0.262875,-0.018311,0.076211,0.030914,-0.078671,0.570251,-0.187783,-0.054606


In [12]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [13]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.112880,-0.183200,0.216299,0.254406,-0.144071,0.337477,0.332848,-0.178598,-0.209764,0.087514,...,-0.168076,0.243658,-0.121029,-0.169716,0.076209,-0.163776,Abnormal,P1,FP-A,AB02
1,-0.076711,-0.147822,0.461249,0.225885,-0.193500,0.138018,0.472570,-0.209146,-0.204889,-0.019481,...,-0.194916,0.395928,0.232385,-0.098065,0.209123,-0.069518,Abnormal,P1,FP-A,AB04
2,0.757314,-0.155397,0.308257,0.745639,-0.176845,0.635605,0.386733,-0.157212,-0.200776,0.120661,...,-0.192254,0.285801,0.524784,-0.154987,0.066375,-0.181024,Abnormal,P1,FP-A,AB06
3,-0.084495,0.048628,0.265085,0.121334,-0.187050,0.025729,0.104737,-0.067526,-0.168360,0.039968,...,-0.106197,-0.109417,-0.192129,-0.012116,-0.053060,-0.124021,Abnormal,P1,FP-A,AB01
4,0.285680,-0.117711,0.001491,0.253031,-0.234893,-0.061371,0.075303,-0.228541,-0.224167,-0.044950,...,-0.169264,-0.098076,-0.038218,-0.140207,-0.070978,-0.144560,Abnormal,P1,FP-A,AB02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7387,-0.072310,0.340018,-0.024193,0.037278,0.411343,-0.083988,-0.120341,0.151559,0.555141,-0.174963,...,0.446409,-0.181699,-0.175733,0.433471,-0.087821,0.291353,Normal,P8,FP-E,Normal
7388,-0.092394,0.355854,-0.138060,-0.116591,0.532154,-0.201658,-0.204159,0.209273,0.526084,-0.185444,...,0.268936,-0.162285,-0.206996,0.732153,-0.185587,0.376708,Normal,P8,FP-E,Normal
7389,-0.161433,0.409080,-0.182008,0.004676,0.332312,-0.157609,-0.202943,0.292184,0.553160,-0.200821,...,0.835365,-0.212773,-0.180168,0.497360,-0.147703,0.537689,Normal,P8,FP-E,Normal
7390,-0.024467,0.598482,-0.141886,-0.126161,0.181843,-0.185020,-0.187077,0.210121,0.048996,-0.164328,...,0.076211,0.030914,-0.078671,0.570251,-0.187783,-0.054606,Normal,P8,FP-E,Normal


In [15]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVtrain_AN_RF_AN_fold1_3.csv') #เปลี่ยนชื่อไฟล์